In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import math
from tensorflow import keras
from keras import Sequential
from keras.utils import Sequence
from keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D, BatchNormalization, Activation, Dropout, Flatten, Dense, Layer, Embedding
from attention import Attention

In [18]:
def get_embedding_wgt(embeddings_df):
    embeddings_wgts = embeddings_df.to_numpy()
    embeddings_mask = np.zeros((embeddings_wgts.shape[0] + 1, embeddings_wgts.shape[1]), dtype=embeddings_wgts.dtype)
    embeddings_mask[1:,] = embeddings_wgts
    return embeddings_mask

In [4]:
import keras.backend as K
# Add attention layer to the deep learning network
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)
 
    def build(self,input_shape):
        self.W=self.add_weight(name='attention_weight', shape=(input_shape[-1],1), 
                               initializer='random_normal', trainable=True)
        self.b=self.add_weight(name='attention_bias', shape=(input_shape[1],1), 
                               initializer='zeros', trainable=True)        
        super(attention, self).build(input_shape)
 
    def call(self,x):
        # Alignment scores. Pass them through tanh function
        e = K.tanh(K.dot(x,self.W)+self.b)
        # Remove dimension of size 1
        e = K.squeeze(e, axis=-1)   
        # Compute the weights
        alpha = K.softmax(e)
        # Reshape to tensorFlow format
        alpha = K.expand_dims(alpha, axis=-1)
        # Compute the context vector
        context = x * alpha
        context = K.sum(context, axis=1)
        return context

In [5]:
# load data
train_path = '../resources/datasets/StanfordSentimentTreebank\split\SST2Processed2-train.csv'
test_path = '../resources/datasets/StanfordSentimentTreebank\split\SST2Processed2-test.csv'
embeddings_path = '../resources/datasets/StanfordSentimentTreebank\embeddings\SST2Processed2-train.csv'
train_dataframe = pd.read_csv(train_path)
test_dataframe  = pd.read_csv(test_path)

In [42]:
import io
def load_vectors(fname, vocab_dicts):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    print(n, d)
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        try :
            aux = vocab_dicts[tokens[0]]
            data[tokens[0]] = [float(tkn) for tkn in tokens[1:]]
        except KeyError :
            pass
    return data

In [43]:
embeddings_df = pd.read_csv(embeddings_path, index_col=0)
embeddings_wgts = get_embedding_wgt(embeddings_df)
embeddings_wgts.shape

(15371, 300)

In [47]:
orig_embed_dict = {row.name : list(row) for _, row in embeddings_df.iterrows()}
pretrained_embedings_dict = load_vectors("../resources/pretrained-embeddings/wiki-news-300d-1M.vec", orig_embed_dict)

999994 300


In [48]:
len(orig_embed_dict), len(pretrained_embedings_dict)

(15370, 14460)

In [51]:
final_embeddings = orig_embed_dict.copy()
for k, v in pretrained_embedings_dict.items() :
    final_embeddings[k] = v

In [54]:
final_embeddings_df = pd.DataFrame(final_embeddings).T

In [55]:
final_embeddings_wgt = get_embedding_wgt(final_embeddings_df)

In [25]:
model = Sequential()
model.add(Embedding(input_dim=final_embeddings_wgt.shape[0], output_dim=final_embeddings_wgt.shape[1], weights=[final_embeddings_wgt], trainable=False, input_length=1))
# block 1 - convolutional (con1d)layer with max pooling and dropout 
model.add(Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPooling1D(pool_size=2, padding='same'))
# model.add(Dropout(0.2))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'))
model.add(Conv1D(filters=16,  kernel_size=3, activation='relu', padding='same'))
model.add(GlobalAveragePooling1D())
# model.add(Dropout(0.2))

# attention layer
# model.add(attention())

# output layer
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
# model.add(Dropout(0.1))
model.add(Dense(units=1, activation='sigmoid'))

# compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 1, 768)            11804928  
                                                                 
 conv1d_12 (Conv1D)          (None, 1, 128)            295040    
                                                                 
 conv1d_13 (Conv1D)          (None, 1, 64)             24640     
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 1, 64)            0         
 1D)                                                             
                                                                 
 conv1d_14 (Conv1D)          (None, 1, 32)             6176      
                                                                 
 conv1d_15 (Conv1D)          (None, 1, 16)             1552      
                                                      

In [7]:
def levenshtein(source:str, target:str) -> int :
    n = len(source)
    m = len(target)
    
    D = np.zeros((n + 1, m + 1), dtype=int)
    for i in range(1, n + 1) :
        D[i][0] = D[i - 1][0] + 1
    for j in range(1, m + 1) :
        D[0][j] = D[0][j - 1] + 1

    subst_cost = lambda x, y : 0 if x == y else 4
    for i in range(1, n + 1) :
        for j in range(1, m + 1) :
            D[i][j] = min([D[i - 1][j    ] + 1,
                           D[i - 1][j - 1] + subst_cost(source[i - 1], target[j - 1]),
                           D[i    ][j - 1] + 1])
    return D[n][m]
def distance(source:str, target:str) -> float :
    lev = levenshtein(source, target)
    if source in target or target in source : lev = lev - .5 
    return lev

In [12]:
class DatasetSequence(Sequence) :
    def __init__(self, 
                 dataframe:pd.DataFrame, 
                 vocabulario:list[str],
                 translation_dict:dict[str,str]=None,
                 text_column:str='text',
                 label_column:str='class',
                 n_classes:int=1,
                 batch_size:int=16,
                 shuffle:bool=True) :
        self.dataframe = dataframe
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.vocabulario_original = vocabulario
        self.vocabulario_dict = {word : i for i, word in enumerate(self.vocabulario_original)}
        
        vocabulario_real = []
        for txt in list(self.dataframe[text_column]) :
            vocabulario_real += txt.split()
        vocabulario_real = list(pd.Series(vocabulario_real).unique())
        
        if translation_dict is None :
            self.new_words = {}
        else :
            self.new_words = translation_dict
            for new, translation in translation_dict.items() :
                self.vocabulario_dict[new] = self.vocabulario_dict[translation]
        
        for tkn in vocabulario_real :
            if not tkn in self.vocabulario_dict.keys() :
                voc_sort = list(self.vocabulario_dict.keys())
                voc_sort.sort(key=lambda x : distance(tkn, x))
                print(tkn, "to", voc_sort[0])
                self.new_words[tkn] = voc_sort[0]
                self.vocabulario_dict[tkn] = self.vocabulario_dict[voc_sort[0]]
        print("No. new words:", len(self.new_words))
        
        self.text_column = text_column
        self.label_column = label_column
        self.n_classes = n_classes
        self.on_epoch_end()
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(self.dataframe.shape[0])
        if self.shuffle :
            np.random.shuffle(self.indexes)
    def __len__(self) :
        return math.ceil(self.dataframe.shape[0] / self.batch_size)
    def text_processing(self, text:str) :
        tokenlist = text.split()
        
        sequence = []
        for token in tokenlist :
            sequence.append(self.vocabulario_dict[token] + 1)
        
        return sequence
    def __generate_data(self, index) :
        if type(index) == int :
            sequence = self.text_processing(self.dataframe.iloc[self.indexes[index]][self.text_column])
            label = [1 if self.dataframe.iloc[index][self.label_column] >= .5 else 0]
            return np.array(sequence, dtype=int), np.array(label, dtype=int)
        elif type(index) == slice :
            start = index.start
            if start == None : start = 0
            stop = index.stop
            if stop == None or stop > self.dataframe.shape[0] : stop = self.dataframe.shape[0]
            
            max_len = 0
            X, Y = [], []
            for i in range(start, stop) :
                x, y = self.__generate_data(i)
                sentence_len = x.shape[0]
                if sentence_len > max_len : max_len = sentence_len
                X.append(x)
                Y.append(y)

            np_X = np.zeros((len(X), max_len), dtype=int)
            for i, x in enumerate(X) :
                np_X[i, : len(x)] = np.array(x)
            np_Y = np.array(Y, dtype=int)
            return np_X, np_Y
        else :
            raise IndexError()
    def __getitem__(self, index) :
        return self.__generate_data(slice(index * self.batch_size, (index + 1) * self.batch_size))

In [26]:
dataset = DatasetSequence(train_dataframe, embeddings_df.index.to_list())

No. new words: 0


In [27]:
model.fit(dataset, epochs=100)

Epoch 1/100
534/534 [==============================] - 6s 11ms/step - loss: 0.6929 - accuracy: 0.5149
Epoch 2/100
534/534 [==============================] - 5s 10ms/step - loss: 0.6931 - accuracy: 0.5225
Epoch 3/100
534/534 [==============================] - 5s 10ms/step - loss: 0.6923 - accuracy: 0.5229
Epoch 4/100
534/534 [==============================] - 5s 10ms/step - loss: 0.6928 - accuracy: 0.5247
Epoch 5/100
534/534 [==============================] - 5s 10ms/step - loss: 0.6921 - accuracy: 0.5275
Epoch 6/100
534/534 [==============================] - 6s 10ms/step - loss: 0.6922 - accuracy: 0.5275
Epoch 7/100
534/534 [==============================] - 6s 11ms/step - loss: 0.6926 - accuracy: 0.5241
Epoch 8/100
534/534 [==============================] - 6s 11ms/step - loss: 0.6922 - accuracy: 0.5275
Epoch 9/100
534/534 [==============================] - 6s 11ms/step - loss: 0.6923 - accuracy: 0.5275
Epoch 10/100
506/534 [===========================>..] - ETA: 0s - loss: 0.6917 - a

KeyboardInterrupt: 